In [1]:
import pandas as pd
import os
import glob
import numpy as np


In [3]:
# 데이터프레임 이름은 df로 설정
# 데이터 모든컬럼 헤더데이터 통합
df = pd.read_csv("../../data/통합데이터/Final_merged_all_data.csv")

In [ ]:
cols = df.columns

In [ ]:
from scipy.stats import linregress

# df = pd.read_csv("Final_merged_all_data.csv") 

# -------------------------------------------------------
# 1. 소비 감소 기울기 (Trend Slope) - 잔액 데이터 활용
# -------------------------------------------------------
# Columns.txt에 있는 정확한 명칭으로 매핑
cols_balance = [
    '잔액_일시불_B2M', '잔액_할부_B2M', 
    '잔액_일시불_B1M', '잔액_할부_B1M', 
    '잔액_일시불_B0M', '잔액_할부_B0M'
]

# 결측치 0 처리
for col in cols_balance:
    if col in df.columns:
        df[col] = df[col].fillna(0)
    else:
        print(f"⚠️ 경고: {col} 컬럼이 없습니다.")

# 월별 총 잔액 계산
df['Balance_Total_B2M'] = df['잔액_일시불_B2M'] + df['잔액_할부_B2M']
df['Balance_Total_B1M'] = df['잔액_일시불_B1M'] + df['잔액_할부_B1M']
df['Balance_Total_B0M'] = df['잔액_일시불_B0M'] + df['잔액_할부_B0M']

# 기울기 계산 함수
def calc_balance_slope(row):
    y = np.array([row['Balance_Total_B2M'], row['Balance_Total_B1M'], row['Balance_Total_B0M']])
    x = np.array([0, 1, 2]) 
    
    if np.sum(y) == 0: return 0
    
    slope, _, _, _, _ = linregress(x, y)
    mean_val = np.mean(y)
    return slope / mean_val if mean_val != 0 else 0

print("1. 잔액 감소 추세(Slope) 계산 중...")
df['Trend_Slope_Balance'] = df.apply(calc_balance_slope, axis=1)


# -------------------------------------------------------
# 2. 활동성 급감 (Activity Drop) - 이용금액 활용
# -------------------------------------------------------
# 컬럼명 수정: 이용금액_R3M_신용 -> 이용금액_신용_R3M
# 컬럼명 대체: 이용금액_신판_B0M -> 이용금액_신용_B0M (신용=일시불+할부+청구할인후 금액 등)

if '이용금액_신용_R3M' in df.columns and '이용금액_신용_B0M' in df.columns:
    # R3M은 3개월 합계이므로 월평균으로 변환
    df['Avg_Spend_R3M'] = df['이용금액_신용_R3M'] / 3
    
    # 0으로 나누기 방지
    df['Activity_Ratio'] = np.where(df['Avg_Spend_R3M'] > 0, 
                                    df['이용금액_신용_B0M'] / df['Avg_Spend_R3M'], 
                                    1.0)
else:
    print("⚠️ 이용금액 관련 컬럼이 확인되지 않아 Activity_Ratio를 1로 설정합니다.")
    df['Activity_Ratio'] = 1.0


# -------------------------------------------------------
# 3. 위험 점수 (Risk Score) - 수신거부 제외 및 대체
# -------------------------------------------------------
df['Risk_Score'] = 0

# (1) 연체 (50점)
if '연체잔액_B0M' in df.columns:
    df['Risk_Score'] += np.where(df['연체잔액_B0M'] > 0, 50, 0)

# (2) 현금서비스 과다 (20점)
if '잔액_현금서비스_B0M' in df.columns:
    df['Risk_Score'] += np.where(df['잔액_현금서비스_B0M'] > 1000000, 20, 0)

# (3) [대체] 승인 거절 발생 (10점) -> 수신거부 대신 사용
# Columns.txt에 '승인거절건수_B0M' 존재함
if '승인거절건수_B0M' in df.columns:
    df['Risk_Score'] += np.where(df['승인거절건수_B0M'] > 0, 10, 0)


# -------------------------------------------------------
# 4. 최종 타겟 생성
# -------------------------------------------------------
# 조건: 기울기 급락(-0.5 이하) OR 활동성 급감(30% 이하) OR 고위험군(50점 이상)
df['Churn_Probabilistic'] = np.where(
    (df['Trend_Slope_Balance'] <= -0.5) | 
    (df['Activity_Ratio'] <= 0.3) | 
    (df['Risk_Score'] >= 50),
    1, 0
)

print("\n[최종 결과 확인]")
print(df[['발급회원번호', 'Trend_Slope_Balance', 'Activity_Ratio', 'Risk_Score', 'Churn_Probabilistic']].head(10))
print("\n이탈 예측 비율:")
print(df['Churn_Probabilistic'].value_counts(normalize=True))